## Fine tune kontrolü

In [2]:
from pathlib import Path
p = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\third_stage\runs\best_finetune_img896_ep30.pt")
print(p.exists(), p.stat().st_size/1024/1024, "MB")

True 21.53903579711914 MB


In [ ]:
import shutil
from pathlib import Path

src = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Final\runs\B_v8n_finetune_img896_ep30_bs2_AdamW_lr0022\weights\best.pt")
dst_dir = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\final_ware\runs")
dst_dir.mkdir(parents=True, exist_ok=True)

dst = dst_dir / "best_finetune_img896_ep30.pt"   
shutil.copy2(src, dst)

print("COPIED ->", dst)
print("Exists:", dst.exists(), "Size(MB):", dst.stat().st_size/1024/1024)


COPIED -> C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\final_ware\runs\best_finetune_img896_ep30.pt
Exists: True Size(MB): 21.53903579711914


In [ ]:
from pathlib import Path

DATASET_DIR = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced")
DATA_YAML   = DATASET_DIR / "dataset.yaml"

PT_PATH = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\final_ware\runs\best_finetune_img896_ep30.pt")
assert PT_PATH.exists(), f"PT yok: {PT_PATH}"

print("DATA_YAML:", DATA_YAML)
print("PT:", PT_PATH)

DATA_YAML: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\dataset.yaml
PT: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\final_ware\runs\best_finetune_img896_ep30.pt


In [ ]:
from ultralytics import YOLO

model = YOLO(str(PT_PATH))
val_res = model.val(data=str(DATA_YAML), imgsz=896, device=0)
print(val_res)

Ultralytics 8.3.203  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 128.2180.3 MB/s, size: 168.5 KB)
val: Scanning C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\labels\val.cache... 350 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 350/350 349.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 22/22 2.2it/s 10.1s.3s
                   all        350       9159      0.941      0.937      0.967      0.533
Speed: 1.4ms preprocess, 12.5ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to C:\Users\hdgn5\OneDrive\Masast\Head Detection\final_ware\runs\detect\val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matri

In [ ]:
import numpy as np
import pandas as pd
from ultralytics import YOLO
from pathlib import Path

IMG_SIZE = 896
CONF = 0.39         
IOU  = 0.50        
MAX_DET = 300

VAL_IMAGES_DIR = DATASET_DIR / "images" / "val"
VAL_LABELS_DIR = DATASET_DIR / "labels" / "val"

assert VAL_IMAGES_DIR.exists(), VAL_IMAGES_DIR
assert VAL_LABELS_DIR.exists(), VAL_LABELS_DIR

def count_gt_labels(label_path: Path) -> int:
    if not label_path.exists():
        return 0
    lines = [l for l in label_path.read_text(encoding="utf-8").splitlines() if l.strip()]
    return len(lines)

model = YOLO(str(PT_PATH))

rows = []
img_paths = sorted([p for p in VAL_IMAGES_DIR.glob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])

for img_path in img_paths:
    label_path = VAL_LABELS_DIR / (img_path.stem + ".txt")
    gt = count_gt_labels(label_path)

    r = model.predict(
        source=str(img_path),
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        max_det=MAX_DET,
        agnostic_nms=True, 
        device=0,
        verbose=False
    )[0]

    pred = 0 if r.boxes is None else len(r.boxes)

    rows.append({
        "image": img_path.name,
        "gt": gt,
        "pred": pred,
        "err": pred - gt,
        "abs_err": abs(pred - gt)
    })

df = pd.DataFrame(rows)

GT_total = int(df["gt"].sum())
Pred_total = int(df["pred"].sum())
Abs_total_diff = int(abs(Pred_total - GT_total))

MAE = float(df["abs_err"].mean())
RMSE = float(np.sqrt(((df["pred"] - df["gt"])**2).mean()))
mask = df["gt"] > 0
MAPE = float((np.abs(df.loc[mask, "pred"] - df.loc[mask, "gt"]) / df.loc[mask, "gt"]).mean()) if mask.any() else float("nan")

print("N_images:", len(df))
print("GT_total:", GT_total)
print("Pred_total:", Pred_total)
print("Abs_total_diff:", Abs_total_diff)
print("MAE:", MAE)
print("RMSE:", RMSE)
print("MAPE(ignore gt=0):", MAPE)

display(df.sort_values("abs_err", ascending=False).head(20))

N_images: 350
GT_total: 9159
Pred_total: 9024
Abs_total_diff: 135
MAE: 1.4314285714285715
RMSE: 3.460388087236129
MAPE(ignore gt=0): 0.05572031753566841


,image,gt,pred,err,abs_err
160,PartB_00053.jpg,162,115,-47,47
16,PartA_00211.jpg,87,75,-12,12
313,PartB_01990.jpg,29,19,-10,10
297,PartB_01690.jpg,68,58,-10,10
248,PartB_01095.jpg,67,58,-9,9
127,PartA_01690.jpg,83,75,-8,8
142,PartA_01862.jpg,80,72,-8,8
188,PartB_00381.jpg,111,103,-8,8
37,PartA_00537.jpg,74,82,8,8
0,PartA_00039.jpg,67,59,-8,8


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO

DATASET_DIR = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced")
VAL_IMAGES_DIR = DATASET_DIR / "images" / "val"
VAL_LABELS_DIR = DATASET_DIR / "labels" / "val"

PT_PATH = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\final_ware\runs\best_finetune_img896_ep30.pt")
assert PT_PATH.exists(), f"PT yok: {PT_PATH}"
assert VAL_IMAGES_DIR.exists() and VAL_LABELS_DIR.exists()

IMG_SIZE = 896
IOU = 0.50
MAX_DET = 300
DEVICE = 0

CONF_LIST = [0.25, 0.30, 0.33, 0.36, 0.39, 0.42, 0.45]

def count_gt_labels(label_path: Path) -> int:
    if not label_path.exists():
        return 0
    lines = [l for l in label_path.read_text(encoding="utf-8").splitlines() if l.strip()]
    return len(lines)

img_paths = sorted([p for p in VAL_IMAGES_DIR.glob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])
N_images = len(img_paths)

gt_map = {}
GT_total = 0
for img_path in img_paths:
    lp = VAL_LABELS_DIR / (img_path.stem + ".txt")
    g = count_gt_labels(lp)
    gt_map[img_path] = g
    GT_total += g

model = YOLO(str(PT_PATH))

results = []

for conf in CONF_LIST:
    preds = []
    gts = []

    for img_path in img_paths:
        gt = gt_map[img_path]

        r = model.predict(
            source=str(img_path),
            imgsz=IMG_SIZE,
            conf=conf,
            iou=IOU,
            max_det=MAX_DET,
            agnostic_nms=True,
            device=DEVICE,
            verbose=False
        )[0]

        pred = 0 if r.boxes is None else len(r.boxes)

        gts.append(gt)
        preds.append(pred)

    gts = np.array(gts, dtype=np.float32)
    preds = np.array(preds, dtype=np.float32)

    Pred_total = int(preds.sum())
    Abs_total_diff = int(abs(Pred_total - GT_total))

    abs_err = np.abs(preds - gts)
    MAE = float(abs_err.mean())
    RMSE = float(np.sqrt(((preds - gts) ** 2).mean()))
    mask = gts > 0
    MAPE = float((abs_err[mask] / gts[mask]).mean()) if mask.any() else float("nan")

    results.append({
        "conf": conf,
        "N_images": N_images,
        "GT_total": int(GT_total),
        "Pred_total": Pred_total,
        "Abs_total_diff": Abs_total_diff,
        "MAE": MAE,
        "RMSE": RMSE,
        "MAPE(ignore gt=0)": MAPE,
        "bias(Pred-GT)": Pred_total - int(GT_total)
    })

df = pd.DataFrame(results)

df_sorted = df.sort_values(["Abs_total_diff", "MAE"], ascending=[True, True]).reset_index(drop=True)

print("===== CONF SWEEP SUMMARY (sorted) =====")
display(df_sorted)

best = df_sorted.iloc[0].to_dict()
print("\nBEST:", best)


===== CONF SWEEP SUMMARY (sorted) =====


,conf,N_images,GT_total,Pred_total,Abs_total_diff,MAE,RMSE,MAPE(ignore gt=0),bias(Pred-GT)
0,0.33,350,9159,9199,40,1.251429,2.376071,0.057297,40
1,0.36,350,9159,9111,48,1.320000,2.913025,0.055551,-48
2,0.30,350,9159,9282,123,1.288571,2.308370,0.061554,123
3,0.39,350,9159,9024,135,1.431429,3.460388,0.055720,-135
4,0.42,350,9159,8947,212,1.508571,3.727887,0.056788,-212
5,0.25,350,9159,9407,248,1.302857,2.305274,0.066733,248
6,0.45,350,9159,8860,299,1.602857,3.976718,0.055005,-299



BEST: {'conf': 0.33, 'N_images': 350.0, 'GT_total': 9159.0, 'Pred_total': 9199.0, 'Abs_total_diff': 40.0, 'MAE': 1.2514286041259766, 'RMSE': 2.3760712146759033, 'MAPE(ignore gt=0)': 0.057296767830848694, 'bias(Pred-GT)': 40.0}
